### Load from database

In [ ]:
from activity_tracker.pipeline import models, measurement, subject, visit

import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

df_subject = subject.Subject.fetch(format="frame").reset_index()
df_visit = visit.Visit.fetch(format="frame").reset_index()
df_measurement = measurement.DailyMeasurement.fetch(format="frame").reset_index()
df_frailty = models.Frailty.fetch(format="frame").reset_index()

In [ ]:
models.Feature.get_feature_matrix(feature_id=1).columns

### Demographics

In [ ]:
control_count = df_subject[df_subject['group'] == 'control']['subject_id'].nunique()
exercise_count = df_subject[df_subject['group'] == 'exercise']['subject_id'].nunique()

print(f"Unique patients in control group: {control_count}")
print(f"Unique patients in exercise group: {exercise_count}")
print(f"Total unique patients: {control_count + exercise_count}")

In [ ]:
# Measure days per subject
measure_days_df = df_measurement.groupby('subject_id').size().reset_index(name='measured_days').sort_values(by='measured_days', ascending=False)
measure_days_df

In [ ]:
subject.Subject()

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(3, 3, figsize=(12, 12), dpi=100)
cat_vars = [
    ("sex", "Sex"),
    ("ethnicity", "Ethnicity"),
    ("race", "Race"),
    ("marital_status", "Marital Status"),
    ("living_situation", "Living Situation"),
    ("group", "Study Group"),
]

for idx, (col, title) in enumerate(cat_vars):
    ax = axes[idx // 3, idx % 3]
    counts = df_subject[col].value_counts()
    missing_count = df_subject[col].isna().sum()
    sns.barplot(x=counts.index, y=counts.values, ax=ax, palette="Set2")
    ax.set_title(f"{title} (missing # = {missing_count})")
    ax.set_ylabel("Count")
    ax.set_xlabel("")
    if col in ["marital_status", "living_situation"]:
        ax.tick_params(axis='x', rotation=45)
    else:
        ax.tick_params(axis='x', rotation=0)
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='bottom')
    sns.despine(ax=ax)

# Age
age_missing = df_subject["age"].isna().sum()
sns.histplot(df_subject["age"].dropna(), bins=15, kde=True, ax=axes[2, 0], color="#4c72b0")
axes[2, 0].set_title(f"Age (missing # = {age_missing})")
axes[2, 0].set_xlabel("Age")
sns.despine(ax=axes[2, 0])

# Monthly income
income_missing = df_subject["monthly_income"].isna().sum()
sns.histplot(df_subject["monthly_income"].dropna(), bins=15, kde=True, ax=axes[2, 1], color="#dd8452")
axes[2, 1].set_title(f"Monthly Income (missing # = {income_missing})")
axes[2, 1].set_xlabel("Monthly Income")
sns.despine(ax=axes[2, 1])

# Education
education_missing = df_subject["education"].isna().sum()
sns.histplot(df_subject["education"].dropna(), bins=10, kde=False, ax=axes[2, 2], color="#55a868")
axes[2, 2].set_title(f"Education Level (missing # = {education_missing})")
axes[2, 2].set_xlabel("Years of Education")
sns.despine(ax=axes[2, 2])

plt.tight_layout()
plt.show()

### Visit analysis

In [ ]:
plt.figure(figsize=(30, 25), dpi=300)
sns.scatterplot(
    data=df_measurement,
    x='date',
    y='subject_id',
    s=13,
    alpha=0.7,
    label='Measurement'
)
plt.xlabel('Date')
plt.ylabel('Subject ID')
plt.title('Measurement Days')

subjects = sorted(df_measurement['subject_id'].unique())
plt.ylim(subjects[-1], subjects[0])
plt.gca().set_ylim(plt.gca().get_ylim()[0] + 1, plt.gca().get_ylim()[1] - 1)

# Mark visit dates
plt.scatter(
    df_visit['date'],
    df_visit['subject_id'],
    color='red',
    marker='*',
    s=50,
    label='Visit Date'
)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visit intervals
# Add prev visit date and ffp status binary
df_merged = df_frailty.merge(df_visit, on=('subject_id', 'visit_id'), how='left')

df_merged.sort_values(["subject_id", "visit_id"])
df_merged["prev_visit_date"] = df_merged.groupby("subject_id")["date"].shift(1)
df_merged["prev_ffp_status"] = df_merged.groupby("subject_id")["ffp_status"].shift(1)
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['days_since_prev_visit'] = df_merged.groupby('subject_id')['date'].diff().dt.days

visit_ids = sorted([v for v in df_merged['visit_id'].unique() if v != 1])
n_visits = len(visit_ids)

fig, axes = plt.subplots(1, n_visits, figsize=(4 * n_visits, 4), sharey=True)

for i, visit_id in enumerate(visit_ids):
    ax = axes[i]
    group = df_merged[df_merged['visit_id'] == visit_id]
    ax.hist(group['days_since_prev_visit'].dropna(), bins=20, alpha=0.7)
    ax.set_title(f'Visit {visit_id}')
    ax.set_xlabel('Days Since Previous Visit')
    sns.despine(ax=ax)
    if i == 0:
        ax.set_ylabel('Frequency')

plt.tight_layout()
plt.suptitle('# of days between visits', y=1.05)
plt.show()

In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt

visited_subjects = set(df_visit["subject_id"].unique())
measured_subjects = set(df_measurement["subject_id"].unique())
both_subjects = visited_subjects & measured_subjects

# venn diagram
plt.figure(figsize=(5, 5))
venn2([visited_subjects, measured_subjects], set_labels=('Visited', 'Fitbit'))
plt.title('# of subjects: Visited  vs Fitbit')
plt.show()

len(df_measurement["subject_id"].unique()) # number of subjects with fitbit measurements
len(df_visit["subject_id"].unique()) # number of subjects with visits

print(f"{visited_subjects - measured_subjects} was visited but not no fitbit measurement data")

In [ ]:
subject.Subject() & visit.Visit() & "subject_id = 'MDE043'"

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from activity_tracker import utils

# Load mapping
mapping_dict = utils.load_data_mapper("../data/data_mapper.yml")
ffp_labels = mapping_dict['ffp_status']

# Create mapping from string values to numeric for heatmap
status_mapping = {'frail': 0, 'prefrail': 1, 'robust': 2}
df_frailty['ffp_status_numeric'] = df_frailty['ffp_status'].map(status_mapping)

colors = ['r', 'y', 'b']  # frail, prefrail, robust
cmap = ListedColormap(colors)
cmap.set_bad('white')  
bounds = [-0.5, 0.5, 1.5, 2.5]
norm = BoundaryNorm(bounds, cmap.N)

# Create heatmap matrix using numeric values
heat_matrix = df_frailty.pivot(index='subject_id', columns='visit_id', values='ffp_status_numeric')
heat_matrix = heat_matrix.reindex(sorted(heat_matrix.columns), axis=1)

# Plot
plt.figure(figsize=(4, 15))
ax = sns.heatmap(
    heat_matrix,
    cmap=cmap,
    norm=norm,
    linewidths=0.5,
    linecolor='gray',
    cbar=False,
    square=False,
    mask=heat_matrix.isna()
)

# Add a rectangle boundary around the heatmap
ax.add_patch(
    patches.Rectangle(
        (0, 0),
        heat_matrix.shape[1],
        heat_matrix.shape[0],
        fill=False,
        edgecolor='black',
        lw=1.5
    )
)

# Custom legend with dots
legend_labels = [mpatches.Patch(color=colors[i], label=ffp_labels[i]) for i in range(3)]
plt.legend(
    handles=legend_labels,
    bbox_to_anchor=(1.1, 0.95),
    loc='upper left',
    borderaxespad=0.,
    frameon=False
)

plt.title('FFP Status')
plt.xlabel('Visit')
plt.ylabel('Subject ID')
plt.tight_layout()
plt.show()

In [ ]:
# Combine prefrail and robust into no_frail
heat_matrix_no_frail = heat_matrix.replace({1.0: 1, 2.0: 1})
heat_matrix_no_frail = heat_matrix_no_frail.sort_values(by=1)
# For plotting
colors = ['r', 'b']  # frail, no_frail
labels = ['frail', 'no_frail']
cmap = ListedColormap(colors)
cmap.set_bad('white')
bounds = [-0.5, 0.5, 1.5]
norm = BoundaryNorm(bounds, cmap.N)

plt.figure(figsize=(4, 15))
ax = sns.heatmap(
    heat_matrix_no_frail,
    cmap=cmap,
    norm=norm,
    linewidths=0.5,
    linecolor='gray',
    cbar=False,
    square=False,
    mask=heat_matrix_no_frail.isna()
)
# Add a rectangle boundary around the heatmap
ax.add_patch(
    patches.Rectangle(
        (0, 0),
        heat_matrix.shape[1],
        heat_matrix.shape[0],
        fill=False,
        edgecolor='black',
        lw=1.5
    )
)

# Custom legend
legend_labels = [mpatches.Patch(color=colors[i], label=labels[i]) for i in range(2)]
plt.legend(
    handles=legend_labels,
    bbox_to_anchor=(1.1, 0.95),
    loc='upper left',
    borderaxespad=0.,
    frameon=False
)

plt.title('FFP Status (frail vs no_frail)')
plt.xlabel('Visit')
plt.ylabel('Subject ID')
plt.tight_layout()
plt.show()

In [ ]:
# Get the first visit data for each subject
first_visit_data = df_frailty.groupby('subject_id').first().reset_index()

# Count frail vs non-frail at first visit
frail_at_start = (first_visit_data['ffp_status_binary'] == 'frail').sum()
non_frail_at_start = (first_visit_data['ffp_status_binary'] == 'no_frail').sum()
total_subjects = len(first_visit_data)

print(f"Patients who started as FRAIL: {frail_at_start} ({frail_at_start/total_subjects*100:.1f}%)")
print(f"Patients who started as NON-FRAIL: {non_frail_at_start} ({non_frail_at_start/total_subjects*100:.1f}%)")
print(f"Total patients: {total_subjects}")

# Alternative: Show the distribution
print("\nDistribution by starting status:")
print(first_visit_data['ffp_status_binary'].value_counts())

# Show some examples of subjects who started frail vs non-frail
print("\nExamples of subjects who started as FRAIL:")
frail_starters = first_visit_data[first_visit_data['ffp_status_binary'] == 'frail']['subject_id'].head(10)
print(frail_starters.tolist())

print("\nExamples of subjects who started as NON-FRAIL:")
non_frail_starters = first_visit_data[first_visit_data['ffp_status_binary'] == 'no_frail']['subject_id'].head(10)
print(non_frail_starters.tolist())

In [ ]:
import numpy as np

df = df_frailty.merge(df_subject, on='subject_id', how='left')

# Calculate means for ffp_score only
mean_score = df.groupby(['group', 'visit_id'])['ffp_score'].mean().unstack('group').reset_index()

visit_ids = sorted(df['visit_id'].unique(), key=lambda x: int(x))
groups = ['control', 'exercise']
bar_width = 0.35
x = np.arange(len(visit_ids))

fig, ax = plt.subplots(figsize=(6, 4))

for i, group in enumerate(groups):
    ax.bar(x + i*bar_width, mean_score[group], bar_width, label=group, alpha=0.8)

ax.set_title('Mean FFP Score')
ax.set_xlabel('Visit')
ax.set_ylabel('FFP Score')
ax.set_xticks(x + bar_width/2)
ax.set_xticklabels(visit_ids)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid(True, alpha=0.3)

plt.tight_layout()
sns.despine()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Plot using Plotly
ffp_distribution = df.groupby(["visit_id", "ffp_status"]).size().unstack(fill_value=0)

fig = go.Figure()
statuses = df['ffp_status'].unique()
labels = statuses
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for status, label, color in zip(statuses, labels, colors):
    fig.add_trace(go.Bar(
        x=ffp_distribution.index.astype(str),
        y=ffp_distribution[status],
        name=label,
        text=ffp_distribution[status],
        textposition='inside',
        marker_color=color
    ))

fig.update_layout(
    width=800,  
    height=500, 
    title="FFP Status",
    xaxis_title="Visit",
    yaxis_title="Number of Participants",
    barmode='stack',
    legend_title="FFP Status"
)

fig.show()

### State transition

In [ ]:
# Sankey plot of frailty status transitions across visits
import pandas as pd
import plotly.graph_objects as go

# Prepare your data
df_wide = df_frailty.pivot(index='subject_id', columns='visit_id', values='ffp_status_binary')
df_wide = df_wide.sort_index(axis=1) 

labels = ['frail', 'no_frail']
short_labels = {'frail': 'F', 'no_frail': 'N'}

# Node labels: just short label (count)
node_labels = []
for v in range(1, df_wide.shape[1]+1):
    for label in labels:
        count = (df_wide[v] == label).sum()
        node_labels.append(f"{short_labels[label]} ({count})")

sources = []
targets = []
values = []
custom_labels = []

for v in range(1, df_wide.shape[1]): 
    from_states = df_wide[v]
    to_states = df_wide[v+1]
    ct = pd.crosstab(from_states, to_states)
    for i, from_label in enumerate(labels):
        for j, to_label in enumerate(labels):
            count = ct.get(to_label, {}).get(from_label, 0)
            if count > 0:
                sources.append(i + (v-1)*len(labels))
                targets.append(j + v*len(labels))
                values.append(count)
                custom_labels.append(f"{short_labels[from_label]}→{short_labels[to_label]}: {count}")

# Two colors: red for frail, blue for no_frail
color_map = {'frail': 'crimson', 'no_frail': 'royalblue'}
node_colors = []
for v in range(df_wide.shape[1]):
    node_colors.extend([color_map[label] for label in labels])

fig = go.Figure(data=[go.Sankey(
    node=dict(label=node_labels, color=node_colors, pad=40, thickness=40),
    link=dict(source=sources, target=targets, value=values, label=custom_labels)
)])

n_visits = df_wide.shape[1]
for v in range(n_visits):
    x = v / (n_visits - 1) if n_visits > 1 else 0.5
    fig.add_annotation(
        x=x,
        y=-0.15,
        text=f"Visit {v+1}",
        showarrow=False,
        font=dict(size=15),
        xref="paper",
        yref="paper"
    )

fig.show()

In [ ]:
# Assuming df_frailty is already loaded and has 'subject_id', 'visit_id', 'ffp_status_binary'
df = df_frailty[['subject_id', 'visit_id', 'ffp_status_binary']].dropna(subset=['ffp_status_binary'])
df = df.sort_values(['subject_id', 'visit_id'])

transitions = {
    ('frail', 'frail'): 0,
    ('frail', 'no_frail'): 0,
    ('no_frail', 'frail'): 0,
    ('no_frail', 'no_frail'): 0
}

for subject, group in df.groupby('subject_id'):
    states = group['ffp_status_binary'].tolist()
    for i in range(len(states) - 1):
        if pd.notna(states[i]) and pd.notna(states[i+1]):
            transitions[(states[i], states[i+1])] += 1

transition_matrix = pd.DataFrame(
    [[transitions[('frail', 'frail')], transitions[('frail', 'no_frail')]],
     [transitions[('no_frail', 'frail')], transitions[('no_frail', 'no_frail')]]],
    index=['frail', 'no_frail'],
    columns=['frail', 'no_frail']
)
print(transition_matrix)

In [ ]:
import numpy as np
prob_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
print(prob_matrix)

eigvals, eigvecs = np.linalg.eig(prob_matrix.T)
stationary = np.real(eigvecs[:, np.isclose(eigvals, 1)])
stationary = stationary / stationary.sum()
print("Stationary distribution:", stationary.flatten())

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, FancyArrowPatch

# Transition probabilities
probs = {
    ('F', 'F'): 0.630,
    ('F', 'N'): 0.370,
    ('N', 'F'): 0.158,
    ('N', 'N'): 0.842
}

# Node positions
node_pos = {'F': (0, 0), 'N': (3, 0)}

fig, ax = plt.subplots(figsize=(9, 5))
ax.set_xlim(-2, 5)
ax.set_ylim(-2, 2)
ax.axis('off')

# Draw ellipses for nodes
ellipse_F = Ellipse(xy=node_pos['F'], width=1.2, height=1.2,
                    edgecolor='orange', facecolor='#ffe5cc', lw=3)
ellipse_N = Ellipse(xy=node_pos['N'], width=1.2, height=1.2,
                    edgecolor='deepskyblue', facecolor='#e0f7ff', lw=3)
ax.add_patch(ellipse_F)
ax.add_patch(ellipse_N)

# Add node labels
ax.text(*node_pos['F'], 'F', fontsize=28, ha='center', va='center')
ax.text(*node_pos['N'], 'N', fontsize=28, ha='center', va='center')

# Arrow style
arrowprops = dict(arrowstyle='-|>', color='black', lw=2, mutation_scale=20)

# Self-loop F
arrow = FancyArrowPatch((node_pos['F'][0] - 0.2, node_pos['F'][1] + 0.6),
                        (node_pos['F'][0] - 0.6, node_pos['F'][1] + 0.2),
                        connectionstyle="arc3,rad=0.8", **arrowprops)
ax.add_patch(arrow)
ax.text(node_pos['F'][0] - 0.9, node_pos['F'][1] + 0.85,
        f"{probs[('F','F')]:.2f}", fontsize=18)

# Self-loop N
arrow = FancyArrowPatch((node_pos['N'][0] + 0.2, node_pos['N'][1] + 0.6),
                        (node_pos['N'][0] + 0.6, node_pos['N'][1] + 0.2),
                        connectionstyle="arc3,rad=-0.7", **arrowprops)
ax.add_patch(arrow)
ax.text(node_pos['N'][0] + 0.4, node_pos['N'][1] + 0.85,
        f"{probs[('N','N')]:.2f}", fontsize=18)

# F → N
arrow = FancyArrowPatch((node_pos['F'][0] + 0.75, node_pos['F'][1] + 0.15),
                        (node_pos['N'][0] - 0.75, node_pos['N'][1] + 0.15),
                        connectionstyle="arc3,rad=0.0", **arrowprops)
ax.add_patch(arrow)
ax.text(1.5, 0.4, f"{probs[('F','N')]:.2f}", fontsize=18, ha='center')

# N → F
arrow = FancyArrowPatch((node_pos['N'][0] - 0.75, node_pos['N'][1] - 0.15),
                        (node_pos['F'][0] + 0.75, node_pos['F'][1] - 0.15),
                        connectionstyle="arc3,rad=0.0", **arrowprops)
ax.add_patch(arrow)
ax.text(1.5, -0.5, f"{probs[('N','F')]:.2f}", fontsize=18, ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Prepare visit dates for each subject
df_visit['date'] = pd.to_datetime(df_visit['date'])
df_measurement['date'] = pd.to_datetime(df_measurement['date'])

visit_dates = df_visit[['subject_id', 'date', 'visit_id']].copy()

# Function to compute days since last visit for each measurement
def get_days_since_last_visit(row):
    subject_visits = visit_dates[visit_dates['subject_id'] == row['subject_id']]
    past_visits = subject_visits[subject_visits['date'] <= row['date']]
    if past_visits.empty:
        return None
    last_visit_date = past_visits['date'].max()
    return (row['date'] - last_visit_date).days

df_measurement['days_since_last_visit'] = df_measurement.apply(get_days_since_last_visit, axis=1)

In [ ]:
# Plot heatmap of total steps between visits
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

visit_ids = sorted(df_visit['visit_id'].unique())
for i in range(len(visit_ids) - 1):
    v_start = visit_ids[i]
    v_end = visit_ids[i + 1]
    
    # Get visit n and n+1 dates for each subject
    visit_n = df_visit[df_visit['visit_id'] == v_start][['subject_id', 'date']].rename(columns={'date': 'start_date'})
    visit_np1 = df_visit[df_visit['visit_id'] == v_end][['subject_id', 'date']].rename(columns={'date': 'end_date'})
    visit_window = visit_n.merge(visit_np1, on='subject_id', how='inner')
    
    # Filter measurements for this interval
    mask = df_measurement['subject_id'].isin(visit_window['subject_id'])
    filtered = df_measurement[mask].merge(visit_window, on='subject_id', how='left')
    
    # Filter to only include data between  visits
    filtered = filtered[(filtered['date'] >= filtered['start_date']) & (filtered['date'] < filtered['end_date'])]
    
    filtered['days_since_last_visit'] = (filtered['date'] - filtered['start_date']).dt.days
    
    # Pivot table
    heatmap_data = filtered.groupby(['subject_id', 'days_since_last_visit'])['total_steps'].sum().unstack()
    # Note: No fill_value=0 here, so missing values remain NaN
    
    # Apply log transformation
    log_heatmap_data = np.log1p(heatmap_data)
    
    # Plot with mask to show NaN as white
    plt.figure(figsize=(16, max(6, 0.2*log_heatmap_data.shape[0])))
    ax = sns.heatmap(
        log_heatmap_data,
        cmap='viridis',
        cbar_kws={'label': 'log(1 + Total Steps)'},
        linewidths=0.1,
        mask=log_heatmap_data.isna()  # This makes NaN values appear as white
    )
    plt.title(f'Total Steps Between Visit {v_start} and {v_end}')
    plt.xlabel('Days Since Last Visit')
    plt.ylabel('Subject ID')
    plt.tight_layout()
    plt.show()


In [ ]:
subject_id = "MDE003"
df_visit.query(f"subject_id == '{subject_id}'")

In [ ]:
df_measurement.query(f"subject_id == '{subject_id}'")

In [ ]:
df_measurement.query(f"subject_id == '{subject_id}'")

In [ ]:
heatmap_data

In [ ]:
## TODO: Data quality check needed
subject_id = "MDE008"
df_visit.query(f"subject_id == '{subject_id}'")

In [ ]:
df_measurement.query(f"subject_id == '{subject_id}'").sort_values("date")
